In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
from itertools import product
import pandas as pd
import seaborn as sns

pq0, alfa = 0.01 , 0.4

In [121]:
def constroe_estados(profundidade,frota):
    aux = []
    for i in range(profundidade+1):
        for j in range(i+1):
            aux.append((i,j))
    
    estados = list(product(*[aux for i in range(frota)]))
    estados.append(tuple([(-1,-1) for i in range(frota)]))
    acoes = list(product(*[[0,1,2] for i in range(frota)]))
    return estados,acoes

def proximos(estado,acao,prof):
    frota = len(acao)
    aux = [[]]*frota
    pr_ax = [[]]*frota
    rc_ax = [[]]*frota
    cnd = [[]]*frota
    for j,v in enumerate(acao):
        x,y = estado[j][0]+1,estado[j][1]+1
        rec = recompensa([x,y])
        cnd[j] = int(estado[j][0]>=prof)
        if v == 0:
            aux[j] = [(x,y),(x,0)]
            aux_q = pq0/(pq0+(1-pq0)*np.exp(-alfa*estado[j][1]))
            pr_ax[j] = [1-aux_q,aux_q]
            rc_ax[j] = [rec[0],rec[1]]
        elif v == 1:
            aux[j] = [(x,0)]  
            pr_ax[j] = [1]
            rc_ax[j] = [rec[2]]
        elif v == 2:
            aux[j] = [(0,0)]
            pr_ax[j] = [1]
            rc_ax[j] = [rec[3]]

    nest = list(product(*aux))
    probs = list(product(*pr_ax))
    recs = list(product(*rc_ax))
    probs = [np.prod(pr) for pr in probs]
    recs = [np.sum(rc) for rc in recs]
    
    if sum(cnd):
        nest = [tuple([(-1,-1) for i in range(frota)])]
        probs = [1]
        recs = [0]

    if estado[0][1]==-1:
        nest = [tuple([(-1,-1) for i in range(frota)])]
        probs = [1]
        recs = [0]

    return {"estados":nest,"probs":probs,"recs":recs}

def recompensa(estado):
    tvida = estado[0]

    valor_equipamento = np.exp(-0.005*tvida)
    valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)
    valor_defeito = -2*valor_manutencao/valor_equipamento
        
    res_a0_op,res_a0_de = 0.01/valor_equipamento,valor_defeito

    res_a1 = -valor_manutencao/valor_equipamento
    
    res_a2 = -(1 - 0.8*valor_equipamento)
    
    res = [res_a0_op,res_a0_de,res_a1,res_a2]
    if estado[0]==-1:
        res = [0,0,0,0]
    return res

def cria_politica(lista_estados):
    frota = len(lista_estados[0])
    politica = {}
    for est in lista_estados:
        aux = [0]*(3**frota)
        aux[0] = 1
        politica[est] = aux
    return politica

def avalia_politica(sims, lista_estados,acoes,politica):
    tol = 1e-5
    frota = len(lista_estados[0])
    prof = max([est[0][0] for est in lista_estados])

    VN = {v:1-2*random.random() for v in lista_estados}
    VN[tuple([(-1,-1) for i in range(frota)])] = 0
    
    cnt, nor = 0, 10
    
    M = {}
    M[0] = np.array(list(VN.values()))
    val_error = []
    while nor>tol and cnt <= sims:
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            pols = politica[est]
            v_aux = 0
            for aci,acv in enumerate(acoes):
                next = proximos(est,acv,prof)
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                for j_e in range(len(nest)):  
                    v_aux = v_aux + pols[aci]*probs[j_e]*(recs[j_e]+V[nest[j_e]])
           
            VN[est] = v_aux
        
        cnt = cnt+1
        M[cnt%2] = np.array(list(VN.values()))

        # nor = np.linalg.norm(M[cnt]-M[cnt-1])
        nor = np.linalg.norm(M[1]-M[0])
        val_error.append(nor)
    return M

def aprimora_politica(lista_estados,politica,sims = 50):
    # escolhe a melhor política deterministica
    tol = 1e-5
    prof = lista_estados[-2][0]
    
    VN = {v:1-2*random.random() for v in lista_estados}
    VN[(-1,-1)]=0
    
    # politica = {est:0 for v in lista_estados}

    P,M = {},{}
    PLN = {k:v for k,v in politica.items()}

    P[0] = politica
    
    cnt_pol = 0

    estavel = False

    while not estavel and cnt_pol<500:
        cnt_pol = cnt_pol+1
        
        PL = {k:v for k,v in PLN.items()}
        PLN = {}
        
        cnt, nor = 0, 10
        while nor>tol and cnt <= sims:
            
            V = {k:v for k,v in VN.items()}
            VN = {}

            for est in lista_estados:
                next = proximos(est,prof) 
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                pols = politica[est]

                v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
                v_a1 = probs[2]*(recs[2]+V[nest[2]])
                v_a2 = probs[3]*(recs[3]+V[nest[3]])

                VN[est] = pols[0]*v_a0+pols[1]*v_a1+pols[2]*v_a2
            
            cnt = cnt+1
            nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
        
        V = {k:v for k,v in VN.items()}
        M[cnt_pol-1] = {k:v for k,v in VN.items()}
        mud = 0
        for est in lista_estados:
            next = proximos(est,prof) 
            nest = next["estados"]
            probs = next["probs"]
            recs = next["recs"]
            pols = politica[est]
            v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
            v_a1 = probs[2]*(recs[2]+V[nest[2]])
            v_a2 = probs[3]*(recs[3]+V[nest[3]])
            idx_pol_max = np.argmax([v_a0,v_a1,v_a2])
            aux_pol = [0,0,0]
            aux_pol[idx_pol_max]=1
            PLN[est] = aux_pol
            
            if not pols[idx_pol_max] == 1:
                mud = mud+1

        if mud > 0:
            estavel = False
        else:
            estavel = True
            
        politica = {k:v for k,v in PLN.items()}
        P[cnt_pol] = {k:v for k,v in PLN.items()}

    cnt, nor = 0, 10
    while nor>tol and cnt <= sims:
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            next = proximos(est,prof) 
            nest = next["estados"]
            probs = next["probs"]
            recs = next["recs"]
            pols = politica[est]
            
            v_a0 = probs[0]*(recs[0]+V[nest[0]])+probs[1]*(recs[1]+V[nest[1]])
            v_a1 = probs[2]*(recs[2]+V[nest[2]])
            v_a2 = probs[3]*(recs[3]+V[nest[3]])

            VN[est] = pols[0]*v_a0+pols[1]*v_a1+pols[2]*v_a2

        cnt = cnt+1
        nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
    
    M[cnt_pol] = {k:v for k,v in VN.items()}
    
    return P,M

def value_iteration(lista_estados,acoes,politica,sims = 500):
    # escolhe a melhor política deterministica

    tol = 1e-5
    frota = len(lista_estados[0])
    prof = max([est[0][0] for est in lista_estados])

    est0 = lista_estados[0]
    if not isinstance(politica[est0],list):
        frota = len(est0)
        len_ac = 3**frota
        for esta in politica.keys():
            aux = [0]*len_ac
            aux[politica[esta]] = 1
            politica[esta] =  aux

    VN = {v:0.05 for v in lista_estados}
    VN[tuple([(-1,-1) for i in range(frota)])] = 0
    

    PLN = {k:np.argmax(v) for k,v in politica.items()}
    
    M,P = {},{}
    M[0] = np.array(list(VN.values()))
    P[0] = {k:v for k,v in PLN.items()}
    
    cnt, nor = 0, 10
    val_norm = []
    while nor>tol and cnt <= sims:
        
        PL = {k:v for k,v in PLN.items()}
        PLN = {}
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            v_aux = []
            for aci,acv in enumerate(acoes):
                next = proximos(est,acv,prof)
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                v_aux2 = 0
                for j_e in range(len(nest)):  
                    v_aux2 = v_aux2 + probs[j_e]*(recs[j_e]+V[nest[j_e]])
                v_aux.append(v_aux2)

            VN[est] = np.max(v_aux)
            PLN[est] = np.argmax(v_aux)
            
        cnt = cnt+1
        nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
        val_norm.append(nor)
        print(f"sim: {cnt},norma: {nor}, condicao: {nor>tol}",end='\r')
        M[cnt%2] = np.array(list(VN.values()))
        P[cnt%2] = {k:v for k,v in PLN.items()}
    
    return {"politica":P, "valor":M, "norma":val_norm}

def value_iteration2(lista_estados,acoes,politica,sims = 500):
    # escolhe a melhor política deterministica
    
    demanda = 1
    tol = 1e-5
    frota = len(lista_estados[0])
    prof = max([est[0][0] for est in lista_estados])

    est0 = lista_estados[0]
    if not isinstance(politica[est0],list):
        frota = len(est0)
        len_ac = 3**frota
        for esta in politica.keys():
            aux = [0]*len_ac
            aux[politica[esta]] = 1
            politica[esta] =  aux

    VN = {v:0.05 for v in lista_estados}
    VN[tuple([(-1,-1) for i in range(frota)])] = 0
    

    PLN = {k:np.argmax(v) for k,v in politica.items()}
    
    M,P = {},{}
    M[0] = np.array(list(VN.values()))
    P[0] = {k:v for k,v in PLN.items()}
    
    cnt, nor = 0, 10
    val_norm = []
    while nor>tol and cnt <= sims:
        
        PL = {k:v for k,v in PLN.items()}
        PLN = {}
        
        V = {k:v for k,v in VN.items()}
        VN = {}

        for est in lista_estados:
            v_aux = []
            for aci,acv in enumerate(acoes):
                aux_a = sum([int(x==0) for x in acv])
                next = proximos(est,acv,prof)
                nest = next["estados"]
                probs = next["probs"]
                recs = next["recs"]
                v_aux2 = 0
                if aux_a>=demanda:
                    for j_e in range(len(nest)):  
                        v_aux2 = v_aux2 + probs[j_e]*(recs[j_e]+V[nest[j_e]])
                    v_aux.append(v_aux2)
                else:
                    for j_e in range(len(nest)):  
                        v_aux2 = v_aux2 + probs[j_e]*(-10+V[nest[j_e]])
                    v_aux.append(v_aux2)


            VN[est] = np.max(v_aux)
            PLN[est] = np.argmax(v_aux)
            
        cnt = cnt+1
        nor = np.linalg.norm(np.array(list(V.values()))-np.array(list(VN.values())))
        val_norm.append(nor)
        print(f"sim: {cnt:2d}, norma: {nor:4.2f}, condicao: {nor>tol}",end='\r')
        # M[cnt%2] = np.array(list(VN.values()))
        # P[cnt%2] = {k:v for k,v in PLN.items()}
        M = np.array(list(VN.values()))
        P = {k:v for k,v in PLN.items()}
    
    return {"politica":P, "valor":M, "norma":val_norm}

def gera_episodio(T,acoes,politica,est0):
    prof = max([v[0][0] for v in politica.keys()])
    
    if not isinstance(politica[est0],list):
        frota = len(est0)
        len_ac = 3**frota
        for esta in politica.keys():
            aux = [0]*len_ac
            aux[politica[esta]] = 1
            politica[esta] =  aux
        
    est = tuple([x for x in est0])
    t = 0
    ES, AC, RC =[], [], []
    
    while t<T and est[0][0]>-1:
        t = t+1
        polis = np.cumsum(politica[est])
        aux_ac = random.random()
        ac = np.sum([int(x<aux_ac) for x in polis])
        nxt = proximos(est,acoes[ac],prof)
        ee = random.random()
        aux_ee = np.sum([int(x<ee) for x in np.cumsum(nxt["probs"])])
        nest = nxt["estados"][aux_ee]
        rec = nxt["recs"][aux_ee]
        ES.append(est)
        AC.append(acoes[ac])
        RC.append(rec)
        est = tuple([x for x in nest])

    
    return {'estados':ES,'acoes':AC,'recs':RC}

def FirstVisitMC(T,est0,prof,frota,n_eps = 1000,pols=[]):
    
    lista_estados, lista_acoes = constroe_estados(prof,frota)
    
    # pols = cria_politica(lista_estados)
    # pols = {}
    # for est in lista_estados:
    #     pols[est] = [0.9,0.1,0.0]

    if pols==[]:        
        pols = {}
        for est in lista_estados:
            aux = [0]*(3**frota)
            aux[0] = 1
            pols[est] = aux
    
    N = {v:0 for v in lista_estados}
    R = {v:0 for v in lista_estados}
    V = {v:0 for v in lista_estados}
    
    for n in range(n_eps):
        epi = gera_episodio(T,lista_acoes,pols,est0)

        estados = [epi['estados'][t] for t in range(len(epi["estados"])-1,-1,-1)]
        
        G = 0
        for t in range(len(epi["estados"])-2,-1,-1):
            G = G + epi["recs"][t]
            est = epi["estados"][t]
            if est not in estados[:t]:
                R[est] = R[est] + G
                N[est] = N[est] + 1
                # V[est] = V[est]+(G-V[est])/N[est]
                V[est] = V[est]+0.1*(G-V[est])

    # for est in lista_estados:
    #     V[est] = R[est]/(N[est]+1e-15)               

    return {"valor":V,"nvezes":N,"retorno":R}

def OnPolicyFirstVisitMC(T,est0,prof,frota,n_eps = 1000,pols=[],tol=0.3):
    # escolhe a melhor política deterministica
    lista_estados, lista_acoes = constroe_estados(prof,frota)
    
    lac = len(lista_acoes)
    pols = {}
    lista_estados_acoes = []
    for est in lista_estados:
        aux = [0]*lac
        aux[0] = 1
        pols[est] = [x/sum(aux) for x in aux]
    

    
    N = {}
    # R = {}
    V = {}
    lrate = 1
    for n in range(n_eps):
        est0 = random.choice(lista_estados)
        epi = gera_episodio(T,lista_acoes,pols,est0)

        # estados_acoes = [(epi["estados"][x],epi["acoes"][x]) for x in range(len(epi)-1,-1,-1)]
        estados_acoes = [(epi["estados"][x],epi["acoes"][x]) for x in range(len(epi["estados"]))]
        
        G = 0
        for t in range(len(epi["estados"])-2,-1,-1):
            G = G + epi['recs'][t]
            est = ((epi["estados"][t],epi["acoes"][t]))
            if est not in estados_acoes[:t]:
                try:
                    N[est] = N[est] + 1
                    V[est] = V[est]+lrate*(G-V[est])/N[est]
                except KeyError:
                    N[est] = 1
                    V[est] = G
                vals = [-100]*lac
                for i in range(lac):
                    estx = (est[0],lista_acoes[i])
                    try:
                        vals[i] = V[estx]
                    except:
                        pass
                    
                acm = np.argmax(vals)
                aux_pol = [tol/lac]*lac
                aux_pol[acm] = 1-tol+tol/lac
                pols[est[0]] = aux_pol

    return N,V,pols

In [131]:
prof = 12
est_list,acoes_list = constroe_estados(prof,2)
polis = cria_politica(est_list)
# M = avalia_politica(10,est_list,acoes_list,polis)
R = value_iteration2(est_list,acoes_list,polis)
# x = [a[0] for a,b in zip(P[0].items(),P[].items()) if not a[1]==b[1]]

In [132]:
len(est_list)

8282

In [7]:
prof = 27
est_list,acoes_list = constroe_estados(prof,2)
polis = cria_politica(est_list)
epi = gera_episodio(prof,acoes_list,polis,((3,0),(0,0)))
epi["acoes"]

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0)]

In [ ]:
xx = {k:v for k,v in R["politica"][1].items()}
{k:v for k,v in xx.items() if k[0]==k[1] and v>0}

In [ ]:
epi["acoes"]

In [ ]:
RS = FirstVisitMC(prof,((0,0),(0,0)),prof,2,n_eps = 10000,pols = R["politica"][1])

In [122]:
prof = 27
N,V,P = OnPolicyFirstVisitMC(prof,((0,0),(0,0)),prof,2,n_eps = 5000000,pols=[],tol=0.5)


In [111]:
P[((11,11),(11,11))]

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [130]:
est_list,acoes_list = constroe_estados(prof,2)
seed = (10,10)
for ac in acoes_list:
    try:
        print(f"{ac} => {V[((seed,seed),ac)]} => {N[((seed,seed),ac)]}")
    except:
        pass

(0, 0) => -1.6542630413838864 => 14
(0, 1) => -1.8386955480288427 => 3
(0, 2) => -1.4887626164991918 => 7
(1, 1) => -1.8977447644952128 => 2
(1, 2) => -2.945280390911458 => 1
(2, 0) => -1.728057970065315 => 2
(2, 1) => -2.1396752183848204 => 3
(2, 2) => -1.6966500986809017 => 3
